In [ ]:
refgrid = '/home/jasp/autocls_exp3/pxs_rst/refpxgroups_06x06_4.tif'

mixed_rst = [
    '/home/jasp/autocls_exp3/px_mixed/mixed4_clcp2018.tif',
    '/home/jasp/autocls_exp3/px_mixed/mixed4_cos2018.tif',
    '/home/jasp/autocls_exp3/px_mixed/mixed4_cosc2021.tif',
    '/home/jasp/autocls_exp3/px_mixed/mixed4_ua2018.tif',
]

lastlevel_shp = '/home/jasp/autocls_exp3/pxs/px_sample_5.shp'

lastlevel_rst = '/home/jasp/autocls_exp3/pxs_rst/refpxgroups_06x06_5.tif'

db = 'gthruth_valeflores_v3'
pxtbl = 'gthruth_fishnet'
pxtbl_pk = 'cid'

In [ ]:
import os
import numpy as np

from glass.wenv.grs import run_grass
from glass.pys.oss import fprop
from glass.pys.tm import now_as_str
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.rcls.grs import null_to_value
from glass.rst.alg import grsrstcalc
from glass.rd.rst import rst_to_array
from glass.smp.fish.sql import split_fishnetcells_in_four
from glass.prop.prj import get_epsg
from glass.prop.sql import check_last_id
from glass.wt.shp import df_to_shp
from glass.wt.sql import df_to_db

In [ ]:
# Create GRASS GIS Session
loc = now_as_str(utc=True)
ws = os.path.dirname(lastlevel_shp)

gb = run_grass(ws, location=loc, srs=refgrid)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
# Import Raster's

refgrs = rst_to_grs(refgrid)

mixgrs = [rst_to_grs(r) for r in mixed_rst]

# Null to Zero
for mg in mixgrs:
    null_to_value(mg, 0, as_cmd=True)

# Sum mixed rasters
sum_mix = grsrstcalc(" + ".join(mixgrs), "sum_mixed")

# Get ID's of level 4
mix_ids = grsrstcalc(
    f"if(sum_mixed > 0, {refgrs}, null())",
    fprop(lastlevel_rst, 'fn')
)

# Export raster
grs_to_rst(mix_ids, lastlevel_rst, as_cmd=True, rtype=int)

## Get Fishnet

In [ ]:
mixnum = rst_to_array(lastlevel_rst, with_nodata=False)

epsg = get_epsg(lastlevel_rst)

mix_ids = list(np.unique(mixnum))

whra = ", ".join(map(str, mix_ids))

last_id = check_last_id(db, pxtbl_pk, pxtbl)

subfish = split_fishnetcells_in_four(
    db, pxtbl, {'cid' : 'rootid'},
    'geom', epsg, last_id, 'cid', 'cid',
    whr=f"{pxtbl}.cid = ANY(ARRAY[{whra}])",
    valascol={"level" : "5"}
)

df_to_shp(subfish, lastlevel_shp)
df_to_db(
    db, subfish, pxtbl, append=True,
    col_geom="geom", epsg=epsg
)